In [ ]:
#------------------RESNET50 MODEL-----------------

import cv2
import os
import numpy as np
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense


# SPLIT
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return images

smile = load_images_from_folder(r"C:\Users\InsertPath")
rage= load_images_from_folder(r"C:\Users\InsertPath")
cry= load_images_from_folder(r"C:\Users\InsertPath")
orgasm= load_images_from_folder(r"C:\Users\InsertPath")



X_smile= smile
X_rage= rage
X_cry=cry
X_excitement=excitement

y_cry = np.full(len(cry),0)
y_smile = np.full(len(smile),1)
y_rage= np.full(len(rage),2)
y_excitement= np.full(len(excitement),3)

y= list(y_cry)+list(y_smile)+list(y_rage)+list(y_excitement)
X= X_cry+X_smile+X_rage+X_excitement

X=[i/255 for i in X]


#TRAIN

model= Sequential()
model.add(tf.keras.applications.ResNet50V2(
    include_top=False, classes=4,
    input_shape=(450, 450, 3)))
model.layers[0].trainable = False
model.add(Dropout(0.25))
model.add(Dense(127))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(207))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4,activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(
        np.array(X), np.array(y),
        epochs=5000,
        batch_size=10,
        validation_split=0.15,
        shuffle=True,
        workers=10,
        use_multiprocessing=True)


model.save('model.h5')

In [ ]:
# -----------SEQUENTIAL NEURAL NETWORK------------------

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization

model = Sequential()
model.add(Conv2D(32,kernel_size=2, input_shape=(450, 450, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,kernel_size=2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(127,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(270,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(127,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(270,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,kernel_size=2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
# how many output categories there are

model.compile(loss='sparse_categorical_crossentropy',
              # expects labelEncoded input
              # binary_crossentropy expects binary output
              # categorical_crossentropy expects one-hot-encoded input
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(
        np.array(X), np.array(y),
        epochs=5000,
        batch_size=10,
        validation_split=0.12,
        shuffle=True,
        workers=10,
        use_multiprocessing=True)


model.save('model_nn.h5')

In [1]:
#-----------------VIDEO APPLICATION OF MODEL----------------


import numpy as np
import cv2
import sys
import tensorflow as tf
import time

model = tf.keras.models.load_model(r"C:\Users\InsertPath")  
face_cascade = cv2.CascadeClassifier(r"C:\Users\InsertPath") 
cap = cv2.VideoCapture(0)

emotions = ('sad','happy','angry','excited')

while True:
    ret, img = cap.read()
    frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray=frame[y:y+w,x:x+h]
        roi_gray=cv2.resize(frame,(200,200))  
        img_pixels = np.array(roi_gray)  / 255
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        
        predictions = model.predict(img_pixels)  
        print(predictions)
        max_index = np.argmax(predictions[0])  
        print(max_index)
      
        predicted_emotion = emotions[max_index] + ' '+ str(int(predictions[0][max_index]*100)) +'%'

        cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  


    cv2.imshow('img',img)

    if cv2.waitKey(10) == ord('q'):
        break  

cap.release()
cv2.destroyAllWindows()

[[1.6928214e-14 9.9989164e-01 1.0839961e-04 7.9529359e-24]]
1
[[4.2669389e-13 9.9673837e-01 3.2616109e-03 3.7710446e-24]]
1
[[4.6476617e-10 7.8519850e-11 1.0000000e+00 6.0193901e-24]]
2
[[5.0205902e-09 3.3009089e-09 1.0000000e+00 5.1116870e-25]]
2
[[1.00816522e-09 1.02697655e-07 9.99999881e-01 3.32046808e-26]]
2
[[2.0738037e-08 1.9606492e-05 9.9998033e-01 5.3709393e-23]]
2
[[3.9298943e-06 2.9986407e-04 9.9969625e-01 6.1340067e-22]]
2
[[1.2894268e-08 2.5526468e-02 9.7447348e-01 9.0376443e-23]]
2
[[7.0432726e-09 3.6655605e-02 9.6334434e-01 5.4036410e-25]]
2
[[4.9590292e-09 8.3361783e-07 9.9999917e-01 1.8673870e-25]]
2
[[1.6493323e-04 6.5237546e-05 9.9976987e-01 4.6647248e-25]]
2
[[3.3751960e-05 5.7418513e-01 4.2578110e-01 5.3445484e-23]]
1
[[8.9801583e-10 4.3408986e-17 1.0000000e+00 2.1672672e-32]]
2
[[2.1055364e-06 2.2661418e-15 9.9999785e-01 1.8381613e-31]]
2
[[2.5550270e-11 6.0741883e-19 1.0000000e+00 2.0869433e-34]]
2
[[3.5394541e-09 3.8529665e-21 1.0000000e+00 3.9420505e-36]]
2
[[2.

[[2.6949626e-06 9.9655864e-08 9.9999714e-01 9.0320526e-08]]
2
[[4.8279890e-05 1.3704673e-07 9.9995065e-01 1.0112639e-06]]
2
[[6.3891616e-04 7.2280645e-02 9.2670494e-01 3.7545618e-04]]
2
[[2.3377181e-05 9.9974006e-01 2.2640757e-04 1.0071443e-05]]
1
[[2.0715217e-04 1.6516078e-01 3.1221724e-01 5.2241486e-01]]
3
[[9.1109359e-06 9.9148977e-01 1.0124678e-03 7.4886624e-03]]
1
[[1.1798785e-07 1.1584682e-04 9.9988389e-01 1.3564062e-07]]
2
[[1.0192090e-07 1.0865209e-06 9.9999881e-01 2.5293279e-09]]
2
[[3.2278181e-06 6.8753510e-09 9.9870694e-01 1.2898055e-03]]
2
[[6.3102169e-07 1.0719847e-05 9.4973183e-01 5.0256815e-02]]
2
[[7.27815996e-06 1.14611648e-05 8.82512629e-01 1.17468655e-01]]
2
[[1.9081913e-06 8.6228374e-07 9.9986529e-01 1.3194192e-04]]
2
[[7.6183244e-07 3.6733560e-08 9.9996233e-01 3.6872181e-05]]
2
[[1.40393695e-05 1.07145311e-07 9.99984741e-01 1.05333447e-06]]
2
[[0.00228357 0.98178905 0.00305688 0.01287045]]
1
[[2.2828447e-04 6.7937789e-09 9.9977130e-01 4.7593198e-07]]
2
[[2.9100089e

[[9.5855653e-01 2.0661749e-20 4.1443478e-02 2.7726859e-19]]
0
[[9.5855653e-01 2.0661749e-20 4.1443478e-02 2.7726859e-19]]
0
[[9.9972814e-01 1.9628255e-13 2.7183251e-04 9.3592124e-23]]
0
[[1.5963302e-08 1.3445706e-04 9.9986553e-01 4.2598773e-23]]
2
[[3.6003810e-16 1.0000000e+00 1.8420879e-10 3.5913880e-28]]
1
[[7.9762412e-06 9.9998951e-01 2.4884996e-06 3.0231960e-19]]
1
[[1.0778036e-09 9.9999774e-01 2.2389647e-06 1.9844538e-20]]
1
[[1.8357166e-05 3.9802617e-01 6.0195547e-01 2.8618071e-22]]
2
[[4.1443620e-08 7.3048896e-01 2.6951107e-01 4.1391145e-24]]
1
[[4.4662611e-12 9.8848021e-01 1.1519784e-02 2.3973374e-28]]
1
[[7.9816320e-18 9.9999976e-01 2.2405315e-07 2.1110299e-32]]
1
[[2.6527717e-12 9.9997938e-01 2.0606121e-05 3.3115089e-25]]
1
[[8.6842374e-12 9.9974591e-01 2.5406166e-04 5.4965818e-29]]
1
[[7.7512316e-11 9.4887990e-01 5.1120102e-02 7.2644028e-27]]
1
[[2.2616675e-08 6.8413049e-01 3.1586951e-01 1.1375534e-25]]
1
[[1.3705214e-12 9.9999273e-01 7.2837543e-06 1.3007242e-29]]
1
[[6.1531

In [ ]:
# ----------Testing the Model on Pictures-----------


model=tf.keras.models.load_model(r"C:\Users\InsertPath")

import os

for x in os.listdir(r"C:\Users\InsertPath"):
    img_pred= tf.keras.preprocessing.image.load_img("C:\\Users\\InsertPath" + str(x), target_size=(200,200))
    img_pred= tf.keras.preprocessing.image.img_to_array(img_pred)
    img_pred = np.expand_dims(img_pred, axis=0)

    result= model.predict(img_pred)
    print (result)
    print('Pic', x, 'is:')

    if result[0][0] ==1:
        prediction= 'Happy!'
        
    elif result[0][0]==0:
        prediction= 'Sad!'
        
    elif result[0][0]==2:
        prediction= 'Angry!'
        
    elif result[0][0]==3:
        prediction= 'Excited!'
        
    else:
        prediction= 'Error'

    print(prediction)